# MNIST Training using PyTorch

# Overview

Amazon Web Services (AWS) offers a wide range of tools and functionalities for enterprise and individual developers. Among which, SageMaker is a fully managed machine learning service that allows data scientists and developers to build and train machine learning models, and S3, is a data storage service providing the capability to store the dataset that powers our model training process. In this assigment, we are going to use PyTorch in AWS SageMaker to implement and train a Convolutional Neural Network on the MNIST dataset.

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). 

## Setup

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


Screen shot of the set up

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

## Data
### Getting the data



In [ ]:
from torchvision import datasets, transforms

!pip install --upgrade jupyter_client

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [ ]:
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/pytorch-mnist'
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('S3 path: {}'.format(inputs))


### Convolution Neural Network

You will be writing some code to complete the CNN model to recognise handwritten digits. In the provided `mnist.py` file, there are a few parts that needs to be filled in order for the model to run.


### Part (a) and (b)

First, we need to define the CNN model. We suggest the following architecture:

1. A convolutional layer with 10 output channels
2. A max pool layer with stride equals 2
3. A ReLU activation layer
4. A convolutional layer with 20 output channels
5. A dropout layer
6. A max pool layer with stride equals 2
7. A ReLU activation layer
5. A dropout layer
6. 2 fully connected layers
7. A softmax output layer

Under the part labelled part (a) and (b), fill in the code using the relavant PyTorch functions. (Hint: The following functions are helpful - Conv2d, Dropout2d, Linear, max_pool2d, relu, dropout)


Next, we focus on the `train()` function. After setting up the function, we define an optimizer to start training the model. In part (c), we use batch Stochastic Gradient Descent to optimize our CNN model. Here are the steps we need to accomplish:

1. For each batch, reset the optimizer gradients to 0.
2. Feed the data into the model and generate the output.
3. Compute the loss of the model
4. Back propagate the weights of the model

Finally, we can implement the `test()` function in part (d). To complete the function:

1. Feed the data into the model and generate the output.
2. Compute the negative log likelihood loss.
3. Increment the correct counter if the prediction was correct.









The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as

For example, the script run by this notebook:

!pygmentize mnist.py

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [ ]:
estimator.fit({'training': inputs})

## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Evaluate
We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
estimator.delete_endpoint()